# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
#BE CAREFUL OF API USAGE

#Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
#export csv
weathers_df = pd.read_csv('../output_data/cities.csv')
#show csv
weathers_df.head(10)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [ ]:
#here use lat and long as locations and humidity as weight, set to type float
location = weathers_df[["Lat", "Lng"]].astype(float)
humidity_x = weathers_df["Humidity"].astype(float)

In [ ]:
#we create a heatmap

fig = gmaps.figure()
#heat_layer
heat_layer = gmaps.heatmap_layer(location, weights= humidity_x, dissipating = False, max_intensity = 100, point_radius= 2)
fig.add_layer(heat_layer)
#show 
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#set location to narrow down cities to fit weather conditions
#startwith hilo
location = ['Hilo']
#weather conditions
#cloudiness 
#max temp> 70, less than 80
#windspeed<10
h_df = weathers_df[(weathers_df['Cloudiness'] == 0) & (weathers_df['Max Temp']> 70) & (weathers_df['Max Temp'] < 80) & (weathers_df['Wind Speed']< 9)]
h_df
 

In [ ]:
#heatmap vacations
#location
v_locations = h_df[['Lat', 'Lng']]
#humidity
v_humidity = h_df['Humidity'].astype(float)
v_fig = gmaps.figure()
v_heat_layer = gmaps.heatmap_layer(v_locations, weights= v_humidity, dissipating = False, max_intensity = 100, point_radius= 2)
v_fig.add_layer(v_heat_layer)
#show the results now 
v_fig

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#Store into variable named hotel_df
#hotel_df =[]


In [ ]:
# variable name
hotel_df =[]

#setting paramets to search for hotels within 5000 meters
for city in range(len(h_df["City"])):
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lats = h_df.loc[city]["Lat"]
    lngs = h_df.loc[city]["Lng"]
#coordinates  
    coordinates = f" {lats}{lngs}"
    
#possible parameters:

#location, rankby 500, types, key, language, , name, keyword, 
    params = {
        "location" :  coordinates,
        "rankby"   :  "prominence",
        "language"  :  en,
        "types"    : "lodging",
        "key"      :  g_key
    }
 #google place 
    hotel_req = requests.get(url, params=params).json
    hotel_df.append(hotel_req)["name"]
    
#nearest hotel   
h_df["Nearest Hotel"] = hotel_df
#show results
h_df.head(10)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
#
